<a href="https://colab.research.google.com/github/Okta408/4212210152_Okta-Saputra-Panjaitan_AAS/blob/main/Another_copy_of_program_fuzzzy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. Import Library
# ====================================
!pip install scikit-fuzzy openpyxl

import pandas as pd
import numpy as np
import skfuzzy as fuzz
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.8/920.8 kB 16.6 MB/s eta 0:00:00


In [ ]:
# 2. Baca Data
# ====================================
file_path = '/content/Fish.new.csv'  # Pastikan file di-upload ke Colab
df = pd.read_csv(file_path, sep=';') # Added sep=';' to specify the separator

# Ambil nilai X, Y, Z dan label aktivitas
w_vals = df['Width'].values
x_vals = df['Weight'].values
y_vals = df['Length1'].values
z_vals = df['Height'].values
labels = df['Species'].values



In [ ]:
# 3. Encode Label
# ====================================
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
unique_labels = np.unique(encoded_labels)[:3]  # Gunakan 3 label pertama

# ====================================

In [ ]:
# 4. Tentukan Range Fuzzy Tiap Label
# ====================================
def get_ranges(label_id):
    idx = encoded_labels == label_id
    x_min, x_max = x_vals[idx].min(), x_vals[idx].max()
    y_min, y_max = y_vals[idx].min(), y_vals[idx].max()
    z_min, z_max = z_vals[idx].min(), z_vals[idx].max()
    return {
        "x": (x_min, (x_min + x_max) / 2, x_max),
        "y": (y_min, (y_min + y_max) / 2, y_max),
        "z": (z_min, (z_min + z_max) / 2, z_max)
    }

label_ranges = {label: get_ranges(label) for label in unique_labels}

# ====================================

In [ ]:
# 5. Fuzzifikasi (TRIMF)
# ====================================
def trimf(x, a, b, c):
    return np.maximum(np.minimum((x - a)/(b - a), (c - x)/(c - b)), 0)

fuzzified = {}
for label in unique_labels:
    r = label_ranges[label]
    fuzzified[f"x_label{label}"] = trimf(x_vals, *r["x"])
    fuzzified[f"y_label{label}"] = trimf(y_vals, *r["y"])
    fuzzified[f"z_label{label}"] = trimf(z_vals, *r["z"])

fuzzified_df = pd.DataFrame(fuzzified)

# ====================================

In [ ]:
# 6. Inferensi Mamdani (MIN)
# ====================================
rules = []
for label in unique_labels:
    rule = np.fmin(np.fmin(
        fuzzified_df[f"x_label{label}"],
        fuzzified_df[f"y_label{label}"]),
        fuzzified_df[f"z_label{label}"]
    )
    rules.append(rule)

# ====================================

In [ ]:
# 7. Agregasi dan Defuzzifikasi
# ====================================
rules = np.array(rules)
total = rules.sum(axis=0)
total[total == 0] = 1e-6  # Hindari pembagian nol

crisp_output = np.dot(unique_labels, rules) / total
predicted_labels = np.round(crisp_output).astype(int)

# ====================================

In [ ]:
# 8. Gabungkan Hasil
# ====================================
hasil_akhir = df.copy()
hasil_akhir["prediksi_label"] = predicted_labels
hasil_akhir["prediksi_kategori"] = label_encoder.inverse_transform(predicted_labels)

# ====================================

In [ ]:
# 9. Evaluasi Akurasi
# ====================================
print("Akurasi:", accuracy_score(encoded_labels, predicted_labels))
print(classification_report(encoded_labels, predicted_labels, target_names=label_encoder.classes_))

# Simpan hasil
hasil_akhir.to_excel("hasil_fuzzy_mamdani.xlsx", index=False)

Akurasi: 0.578125
              precision    recall  f1-score   support

       Bream       0.64      0.66      0.65        35
       Perch       0.54      0.80      0.64        56
        Pike       0.75      0.35      0.48        17
       Roach       0.00      0.00      0.00        20

    accuracy                           0.58       128
   macro avg       0.48      0.45      0.44       128
weighted avg       0.51      0.58      0.52       128



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
